In [ ]:
from collections.abc import Generator, Callable
from pathlib import Path
import typing
from typing import Any, TypeAlias, Literal
import pandas as pd
import numpy as np
import datetime as dt
import re
from functools import partial, reduce
from tqdm import tqdm
from IPython.display import (
    display, # type: ignore[reportUnknownVariableType]
    Markdown,
)

import importlib

from config.fastf1 import fastf1
from config import config
import src.data.constants as dataset_constants
importlib.reload(dataset_constants);
import src.data.loader
importlib.reload(src.data.loader);
from src.data.loader import stream_ndjson, load_submissions_df, load_comments_df
from src.data.preprocessing import concatenate_submissions_and_comments

from src.utils import (
    temporary_pandas_options,
    display_full_dataframe,
    hide_index,
    compose,
)
from src import utils
utils.set_random_seeds()
DEVICE = utils.get_device()

import logging
logging.getLogger('fastf1').setLevel(logging.WARNING)

# Load data and find submissions related to steward decisions

In [2]:
f1_submissions_df = load_submissions_df(
    dataset_constants.RawFile.FORMULA1_SUBMISSIONS,
    columns=dataset_constants.DEFAULT_SUBMISSION_COLUMNS | {'permalink', 'post_hint', 'link_flair_text'},
)                                  

f1_comments_df = load_comments_df(
    dataset_constants.RawFile.FORMULA1_COMMENTS,
    columns=dataset_constants.DEFAULT_COMMENT_COLUMNS | {'link_id'},
)

In [3]:
f1_comments_df = f1_comments_df[~f1_comments_df['body'].isin({'[removed]', '[deleted]'})]

In [4]:
f1_submissions_df['permalink'] = 'www.reddit.com' + f1_submissions_df['permalink']

In [ ]:
steward_decision_related_words = {
    'penalty', 'steward', 'decision', 'appeal', 'review', 'ruling', 'investigation', 'regulation',
    'seconds', 'sec', 
    'collision', 'crash', 'incident', 'overtake', 'virtual safety car', 'blocking', 'brake test', 'contact',
    'red flag', 'yellow flag', 
    'controversial', 'rigged', 'corrupt', 'bias', 'protest', 'FIA', 'document', 'infringement'}

# Manually exclude some posts unrelated to steward decisions
excluded_submission_ids = {
    'vdr1c6',
    'w7z5aj',
    'wf87e0',
    'x1zd5z',
    'x3y140',
}

words_regex = ''.join(fr'\b{word}\b|' for word in steward_decision_related_words)[:-1]
steward_decision_pattern = re.compile(words_regex, flags=re.IGNORECASE)

relevant_flairs = {':post-technical: Technical', ':post-news: News'}

has_related_words = f1_submissions_df['title'].apply(lambda title: steward_decision_pattern.search(title) is not None)
has_relevant_flairs = f1_submissions_df['link_flair_text'].isin(relevant_flairs)
is_image_post = f1_submissions_df['post_hint'] == 'image'
is_included = ~f1_submissions_df['id'].isin(excluded_submission_ids) 

steward_decision_submissions_df = f1_submissions_df[has_related_words & has_relevant_flairs & is_image_post & is_included].copy()

with display_full_dataframe():
    print(len(steward_decision_submissions_df))
    display(steward_decision_submissions_df.head(2))

# Discretization of continuous sentiment function

In [6]:
neutral_range = (-0.05, 0.05)

def to_sentiment_category(sentiment: float) -> Literal['Positive', 'Negative', 'Neutral']:
    if sentiment >= neutral_range[1]:
        return 'Positive'
    elif sentiment <= -neutral_range[0]:
        return 'Negative'
    else:
        return 'Neutral'

def to_discrete_sentiment(sentiment: float) -> int:
    category = to_sentiment_category(sentiment)

    match category:
        case 'Positive':
            return 1
        case 'Negative':
            return -1
        case 'Neutral':
            return 0

### VADER SENTIMENT ANALYSIS

In [7]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vader_analyzer = SentimentIntensityAnalyzer()

In [ ]:
for index, steward_decision_submission in steward_decision_submissions_df.iterrows():
    submission_link_id = f't3_{steward_decision_submission['id']}'
    comments_df = f1_comments_df[f1_comments_df['link_id'] == submission_link_id].copy()

    if comments_df.empty:
        steward_decision_submissions_df.loc[index, 'average_sentiment_vader'] = np.nan
        continue
    
    number_of_votes = np.abs(comments_df['score']).sum()
    
    if number_of_votes == 0:
        steward_decision_submissions_df.loc[index, 'average_sentiment_vader'] = np.nan
        continue

    comments_df.loc[:, 'compound'] = comments_df['body'].apply(
        lambda text: vader_analyzer.polarity_scores(text)['compound']
    )

    average_sentiment = (comments_df['compound'] * comments_df['score']).sum() / number_of_votes
    steward_decision_submissions_df.loc[index, 'average_sentiment_vader'] = average_sentiment

with display_full_dataframe():
    display(steward_decision_submissions_df.head(2))

### BERT SENTIMENT ANALYSIS

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model.to(DEVICE);

import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

def bert_sentiment(text: str) -> float:
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512).to(DEVICE)

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    sentiment_score = torch.argmax(logits, dim=1).item()  # 0: negative, 1: neutral, 2: positive

    return (sentiment_score - 1) #-1: negative, 0: neutral, 1: positive

In [10]:
for index, steward_decision_submission in steward_decision_submissions_df.iterrows():
    submission_link_id = f't3_{steward_decision_submission['id']}'
    comments_df = f1_comments_df[f1_comments_df['link_id'] == submission_link_id].copy()

    if comments_df.empty:
        steward_decision_submissions_df.loc[index, 'average_sentiment_bert'] = np.nan
        continue
    
    number_of_votes = np.abs(comments_df['score']).sum()
    
    if number_of_votes == 0:
        steward_decision_submissions_df.loc[index, 'average_sentiment_bert'] = np.nan
        continue

    comments_df.loc[:, 'compound'] = comments_df['body'].apply(bert_sentiment)

    average_sentiment = (comments_df['compound'] * comments_df['score']).sum() / number_of_votes
    steward_decision_submissions_df.loc[index, 'average_sentiment_bert'] = average_sentiment

#  with display_full_dataframe():
    # display(steward_decision_submissions_df)
    # display(steward_decision_submissions_df['average_sentiment_bert'])

# (extra) sentiment analysis between controversial drivers, 
# Verstappen & Alonso

In [ ]:
verstappen_decisions = steward_decision_submissions_df[
    steward_decision_submissions_df['title'].apply(lambda title: re.search(r'max|verstappen', title, flags=re.IGNORECASE) is not None)
]
alonso_decisions = steward_decision_submissions_df[
    steward_decision_submissions_df['title'].apply(lambda title: re.search(r'fernando|alonso', title, flags=re.IGNORECASE) is not None)
]

ignored_alonso_submissions = {'wzw8e1', 'ybswfs'}
alonso_decisions = alonso_decisions[~alonso_decisions['id'].isin(ignored_alonso_submissions)]

print(verstappen_decisions['average_sentiment_bert'].mean())
print(alonso_decisions['average_sentiment_bert'].mean())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting
plt.figure(figsize=(12, 6))
sns.scatterplot(data=steward_decision_submissions_df, x='created_utc', y='average_sentiment_bert', marker='o')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Average Sentiment (BERT)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


# EXTRINSIC VALIDATION - CONFUSION MATRIX

attempt 2

In [ ]:
import json
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

validation_labels_dir = config.ROOT_DIR / 'validation_labels' / 'steward_decision_submissions'
labeled_comments = tuple(
    labeled_comment
    for file in validation_labels_dir.glob('*.ndjson')
    for labeled_comment in stream_ndjson(file)
)

comment_ids = {labeled_comment['comment_id'] for labeled_comment in labeled_comments}

y_true = [labeled_comment['sentiment'] for labeled_comment in labeled_comments]
y_vader = [to_sentiment_category(vader_analyzer.polarity_scores(f1_comments_df.loc[f1_comments_df['id'] == labeled_comments, 'body'].iloc[0])['compound']) for labeled_comment in labeled_comments]
y_bert = [to_sentiment_category(bert_sentiment(f1_comments_df.loc[f1_comments_df['id'] == labeled_comment['comment_id'], 'body'].iloc[0])) for labeled_comment in labeled_comments]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for ax, (y_pred, title) in zip(axes, [(y_vader, "VADER"), (y_bert, "BERT")]):
    cm = confusion_matrix(y_true, y_pred, labels=["Positive", "Neutral", "Negative"])
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=["Positive", "Neutral", "Negative"],
                yticklabels=["Positive", "Neutral", "Negative"], ax=ax)
    ax.set_title(f"Confusion Matrix: {title}")
    ax.set_xlabel("Predicted Label")
    ax.set_ylabel("True Label")

plt.tight_layout()
plt.show()


# accuracy, precision, recall and F1 validation metrics

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_true, y_vader)
print(f"VADER Accuracy: {accuracy:.4f}")
accuracy = accuracy_score(y_true, y_bert)
print(f"BERT Accuracy: {accuracy:.4f}")

labels = ["Positive", "Neutral", "Negative"]
for label in labels:
    vader_precision = precision_score(y_true, y_vader, labels=[label], average="macro", zero_division=0)
    vader_recall = recall_score(y_true, y_vader, labels=[label], average="macro", zero_division=0)
    vader_f1 = f1_score(y_true, y_vader, labels=[label], average="macro", zero_division=0)

    bert_precision = precision_score(y_true, y_bert, labels=[label], average="macro", zero_division=0)
    bert_recall = recall_score(y_true, y_bert, labels=[label], average="macro", zero_division=0)
    bert_f1 = f1_score(y_true, y_bert, labels=[label], average="macro", zero_division=0)

    print(f"\nMetrics for class '{label}':")
    print(f"  VADER -> Precision: {vader_precision:.4f}, Recall: {vader_recall:.4f}, F1-score: {vader_f1:.4f}")
    print(f"  BERT  -> Precision: {bert_precision:.4f}, Recall: {bert_recall:.4f}, F1-score: {bert_f1:.4f}")


In [ ]:
print(f'{steward_decision_submissions_df['average_sentiment_bert'].mean()=}')
print(f'{steward_decision_submissions_df['average_sentiment_bert'].min()=}')
print(f'{steward_decision_submissions_df['average_sentiment_bert'].max()=}')
